# ライブラリのインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

# 動画に保存する関数

In [2]:
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def save_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)

    anim.save('RL-cartpole.mp4')
    display(display_animation(anim, default_mode='loop'))


In [3]:
# 学習に使う変数を整理
ENV = 'CartPole-v0'
# 状態をどれくらいの値で分割して離散値とするか
NUM_DIGITIZED = 6
# 報酬割引率
GAMMA = 0.99
# 学習率
ETA = 0.5
# 1試行（1エピソード）の最大ステップ数
MAX_STEP = 200
# 最大試行回数（エピソード数）
NUM_EPISODES = 500


In [4]:
# エージェントクラス
class Agent:

    # 初期化関数で行動数と行動の種類を受けとり，行動を決定するためのBrainクラスを生成する
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions)

    # Q関数を更新
    def update_q_function(self, observation, action, reward, observation_next):
        self.brain.update_Q_table(observation, action, reward, observation_next)

    # 行動の決定
    def get_action(self, observation, step):
        return self.brain.decide_action(observation, step)


# エージェントが行う行動を与えられた状態によって判断する部分（強化学習を行う部分）
class Brain:

    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        # Qテーブルを作成。行数は状態を分割数^（4変数）にデジタル変換した値、列数は行動数を示す
        self.q_table = np.random.uniform(low=0, high=1, size=(NUM_DIGITIZED**num_states, num_actions))


    # カート情報を離散化するための閾値を求める
    def bins(self, clip_min, clip_max, num):
        return np.linspace(clip_min, clip_max, num + 1)[1:-1]

    # カートの情報を離散化する
    def digitize_state(self, observation):
        '''観測したobservation状態を、離散値に変換する'''
        cart_pos, cart_v, pole_angle, pole_v = observation
        digitized = [
            np.digitize(cart_pos, bins=self.bins(-2.4, 2.4, NUM_DIGITIZED)),
            np.digitize(cart_v, bins=self.bins(-3.0, 3.0, NUM_DIGITIZED)),
            np.digitize(pole_angle, bins=self.bins(-0.5, 0.5, NUM_DIGITIZED)),
            np.digitize(pole_v, bins=self.bins(-2.0, 2.0, NUM_DIGITIZED))
        ]
        return sum([x * (NUM_DIGITIZED**i) for i, x in enumerate(digitized)])

    # QテーブルをQ学習により更新
    def update_Q_table(self, observation, action, reward, observation_next):
        Q = self.q_table
        state = self.digitize_state(observation)
        state_next = self.digitize_state(observation_next)
        Q[state, action] = Q[state, action] + ETA * (reward + GAMMA * np.nanmax(Q[state_next, :]) - Q[state, action])
        return Q
        


    # ε-greedy法で徐々に最適行動を行うようにする
    def decide_action(self, observation, episode):
        state = self.digitize_state(observation)
        epsilon = 0.5 * (1 / (episode+1))
        
        if epsilon < np.random.uniform(0, 1):
            action = np.argmax(self.q_table[state][:])
        else:
            action = np.random.choice(self.num_actions)
        return action


In [5]:
## CartPoleを実行する環境
class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態を取得
        num_actions = self.env.action_space.n  # CartPoleの行動を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

    ## 実行関数
    def run(self):

        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        is_episode_final = False  # 最終試行フラグ
        frames = []  # 動画用に画像を格納する変数

        # 全エピソードループ
        for i in range(NUM_EPISODES):
            # ほげほげ
            observation = self.env.reset()
            episode_reward = 0

            # 1エピソードループ
            for l in range(MAX_STEP):
                if is_episode_final is True:
                    # framesに各時刻の画像を追加していく
                    frames.append(self.env.render(mode='rgb_array'))
            # ほげほげ
                action = self.agent.get_action(observation, i)
                observation_next, reward_notuse, done, info_notuse = self.env.step(action)
                
                if done:
                    if l < 195:
                        reward = -1
                        self.complete_episodes = 0
                    else:
                        reward = 1
                        complete_episodes += 1
                else:
                    reward = 0
                
                episode_reward += reward
                
                self.agent.update_q_function(observation, action, reward, observation_next)
                
                observation = observation_next
                
                if done:
                    print('{0} Episode: Finished after {1} time steps'.format(i, l+1))
                    break
                    
            if is_episode_final is True:
                # 動画を保存と描画
                save_as_gif(frames)
                break

            if complete_episodes >= 10:
                print('10回連続成功')
                frames = []
                is_episode_final = True  # 次の試行を描画を行う最終試行とする

In [6]:
# main
cartpole_env = Environment()
cartpole_env.run()

0 Episode: Finished after 10 time steps
1 Episode: Finished after 12 time steps
2 Episode: Finished after 56 time steps
3 Episode: Finished after 27 time steps
4 Episode: Finished after 18 time steps
5 Episode: Finished after 8 time steps
6 Episode: Finished after 12 time steps
7 Episode: Finished after 44 time steps
8 Episode: Finished after 11 time steps
9 Episode: Finished after 36 time steps
10 Episode: Finished after 11 time steps
11 Episode: Finished after 54 time steps
12 Episode: Finished after 16 time steps
13 Episode: Finished after 15 time steps
14 Episode: Finished after 11 time steps
15 Episode: Finished after 25 time steps
16 Episode: Finished after 9 time steps
17 Episode: Finished after 96 time steps
18 Episode: Finished after 17 time steps
19 Episode: Finished after 35 time steps
20 Episode: Finished after 55 time steps
21 Episode: Finished after 70 time steps
22 Episode: Finished after 78 time steps
23 Episode: Finished after 69 time steps
24 Episode: Finished after 1

AttributeError: 'HTMLWriter' object has no attribute '_temp_names'